In [ ]:
datetime.now().strftime("%Y%m%d-%H%M%S")

/home/vishal/anaconda3/envs/uw_model_play2/bin/python

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import pymysql
import pymssql
import sys

In [ ]:
sys.path.insert(0,'/home/shared/utils')

In [ ]:
from db_utils import *
import kpi_count

In [ ]:
iloans_conn = get_iloans_conn()
predicon_conn = get_predicon_db_conn()
bank_app_conn = get_bank_app_conn()

bank_app - loanid, entered_date, bv decisions

predicon_model - loan_id, entered_date, count of scored applicants, #positive

loans - loanid, lead_time_added, #lender approved

## In a Nutshell

### Extracting Data

In [ ]:
bank_app = kpi_count.extract_bankapp(bank_app_conn)

In [ ]:
model = kpi_count.extract_model(predicon_conn)

In [ ]:
funded_loans = kpi_count.extract_funded_loans(iloans_conn)

### Modifying Data

In [ ]:
md_bnk = kpi_count.modify_bankapp_db(bank_app)

In [ ]:
md_mod = kpi_count.modify_model_db(model)

In [ ]:
md_fl = kpi_count.modify_fundedloans_db(funded_loans)

### Merging all the modified datasets and generating model kpi metrics

In [ ]:
kpi_func = kpi_count.get_kpi(md_bnk, md_mod, md_fl)

In [ ]:
kpi_func